In [1]:
# 1. Get 10,000 posts

In [1]:
import sys
import requests
calais_url = 'https://api.thomsonreuters.com/permid/calais'

import webhoseio, os
from gensim.models import KeyedVectors
import json
from scipy import spatial
from simhash import Simhash, SimhashIndex
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import nltk, re
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, sent_tokenize, ngrams
stopwords = set(nltk.corpus.stopwords.words('english'))

import spacy
import en_core_web_sm

In [2]:
webhoseio.config(token="2c45808b-852d-4156-af2f-faf960992ea9")
query_params = {"q": "organization:apple language:english published:>1533099600000", 
                "sort": "crawled"}
output = webhoseio.query("filterWebContent", query_params)
print(output['posts'][0]['text']) # Print the text of the first post
print(output['posts'][0]['published']) # Print the text of the first post publication date
# Print the site of the first post
print(output['posts'][0]['thread']['site'])

feeds = []
for item in output['posts']:
    feeds.append(item)
    #print(item['title'])
print(len(feeds))

count = 99
while count > 0:    
    output = webhoseio.get_next()
    for item in output['posts']:
        feeds.append(item)
    count -= 1
print(len(feeds)) #try org:google to get 10,000 feeds

with open("anly540_final.json", "w") as myfile: 
    for feed in feeds:
        line = json.dumps(feed)
        myfile.write(line)
        myfile.write("\n")

By Gun Talk Media in Gun Rights
The following video is brought to you courtesy of the Gun Talk Media YouTube Channel. Click the video below to watch it now.
Tom talks hunting ammo and more with Black Hills Ammunition’s Jeff Hoffman. Also, a range report on a 1911 CMP.
Tom Gresham’s Gun Talk® airs live on Sundays from 2PM-5PM Eastern, and runs on hundreds of radio stations nationwide. This gun-centric show features everything from new products to politics. Listen live on a radio station near you or via streaming. Listen to past shows now on Gun Talk’s channels on YouTube, Roku, Amazon Fire and Apple TV, and at guntalk.libsyn.com, Apple iTunes, i-Heart, Stitcher, the GunDealio app for iPhone or Android, and at guntalk.com.
©2018 Gun Talk Radio, LLC. All rights reserved. Recommended Viewing EXCLUSIVE: Claim Your Free 72-Hour Survival Food Kit Now…
In a crisis, your #1 need is food.
But not just any food...
Experts say everyone needs to have non-perishable, good for 25 years survival food 

In [3]:
json_data = open("anly540_final.json").readlines()
feeds_read_from_file = []
for line in json_data:
    feeds_read_from_file.append(json.loads(line))
print(len(feeds_read_from_file))

for feed in feeds_read_from_file[:10]:
    print(feed['title'], '\n')
    print(feed['text'], '\n')
    print(feed['url'], '\n')
#print(feeds_read_from_file[0]['title'])

8019
Hunting Ammo; 1911 CMP Range Report: Gun Talk Radio | 12.9.18 C 

By Gun Talk Media in Gun Rights
The following video is brought to you courtesy of the Gun Talk Media YouTube Channel. Click the video below to watch it now.
Tom talks hunting ammo and more with Black Hills Ammunition’s Jeff Hoffman. Also, a range report on a 1911 CMP.
Tom Gresham’s Gun Talk® airs live on Sundays from 2PM-5PM Eastern, and runs on hundreds of radio stations nationwide. This gun-centric show features everything from new products to politics. Listen live on a radio station near you or via streaming. Listen to past shows now on Gun Talk’s channels on YouTube, Roku, Amazon Fire and Apple TV, and at guntalk.libsyn.com, Apple iTunes, i-Heart, Stitcher, the GunDealio app for iPhone or Android, and at guntalk.com.
©2018 Gun Talk Radio, LLC. All rights reserved. Recommended Viewing EXCLUSIVE: Claim Your Free 72-Hour Survival Food Kit Now…
In a crisis, your #1 need is food.
But not just any food...
Experts say 

In [5]:
# 2. Deduplication

In [6]:
# Loading Word2Vec Google News model
model_path = './'
def load_wordvec_model(modelName, modelFile, flagBin):
    print('Loading ' + modelName + ' model...')
    model = KeyedVectors.load_word2vec_format(model_path + modelFile, binary=flagBin)
    print('Finished loading ' + modelName + ' model...')
    return model

model_word2vec = load_wordvec_model('Word2Vec Google News', 'GoogleNews-vectors-negative300.bin', True)

Loading Word2Vec Google News model...
Finished loading Word2Vec Google News model...


In [7]:
# Original set of Titles and Publish dates
feeds = []
i = 0
for feed in feeds_read_from_file:
    feed['id'] = i
#     if feed['language'] == 'english':
#         print(feed['language'])    
#     print(str(feed['id']), str(feed['title']),  feed['published'])
    i += 1
    feeds.append(feed)

In [8]:
# function checks whether the input words are present in the vocabulary for the model
def vocab_check(vectors, words):
    
    output = list()
    for word in words:
        if word in vectors.vocab:
            output.append(word.strip())
            
    return output

In [9]:
# function calculates similarity between two strings using a particular word vector model
def calc_similarity(input1, input2, vectors):
    s1words = set(vocab_check(vectors, input1.split()))
    s2words = set(vocab_check(vectors, input2.split()))
    
    output = vectors.n_similarity(s1words, s2words)
    return output

In [10]:
def cleanup(input):
    # remove English stopwords
    input = input.replace("'s", " ").replace("n’t", " not").replace("’ve", " have")
    input = re.sub(r'[^a-zA-Z0-9 ]', '', input)
    return input

In [12]:
# Clean identical titles
dups = set()
for i in range(len(feeds)):
    for j in range(i + 1, len(feeds)):
        if j not in dups and feeds[i]['title'].lower() == feeds[j]['title'].lower():
            dups.add(j)
print(len(dups))

2210


In [13]:
# Get unique feeds for further deduplication process
feeds_unique = []
for i, feed in enumerate(feeds):
    if i not in dups:
        feeds_unique.append(feed)

In [14]:
for i, feed in enumerate(feeds_unique):
    feed['id'] = i
# feeds_unique
len(feeds_unique)

5809

In [24]:
# Set up similarity distance as 7 for further deduplication process
distance = 8
objs = [(str(feed['id']), Simhash(str(feed['title']))) for feed in feeds_unique]
index = SimhashIndex(objs, k=distance)
# index.bucket_size()

In [25]:
# indices of duplicated titles
dups = set()
for feed in feeds_unique:
    # Select a feed from the list based on its index
    SELECTED_INDEX = feed['id'] #27
#     if SELECTED_INDEX in dups:
#         continue
    feed_sel = feeds_unique[SELECTED_INDEX]
    # calculate  hash value
    feed_hash = Simhash(str(feed_sel['title']))
    # find all duplicate indices
    dup_indices = index.get_near_dups(feed_hash)
    for dup_index in dup_indices:
        if dup_index != str(SELECTED_INDEX):
            dups.add(dup_index)
            print("Dup#" + str(dup_index) + ": " + feeds_unique[int(dup_index)]['title'])
#     print(dups)
    if len(dup_indices) > 1: 
        print("Original#" + str(SELECTED_INDEX) + ": " + feed_sel['title'])
        print("Number of duplicates (SimHash): " + str(len(dup_indices) - 1))
        print("")

Dup#181: Apple is losing the interest of some big-money investors, RBC says
Original#15: Apple is losing the interest of some big-money investors, RBC says (AAPL)
Number of duplicates (SimHash): 1

Dup#229: Stocks Struggle Higher as Markets Remain Volatile; Oil Drops
Dup#413: Stocks struggle higher as markets remain volatile
Original#39: Stocks struggle higher as markets remain volatile; oil drops - KSL.com
Number of duplicates (SimHash): 2

Dup#4394: Global and Chinese B2B Publishing Industry, 2018 Market Research Report
Dup#4384: Global and Chinese Industrial Internet-of-Things Industry, 2018 Market Research Report
Original#40: Global and Chinese Gaming Industry, 2018 Market Research Report
Number of duplicates (SimHash): 2

Dup#273: China bans most iPhone sales after granting Qualcomm an injunction against Apple - WTVA (0 visits)
Dup#356: China bans most iPhone sales after granting Qualcomm an injunction against Apple
Original#41: China bans most iPhone sales after granting Qualcomm

Dup#41: China bans most iPhone sales after granting Qualcomm an injunction against Apple – CNN
Dup#273: China bans most iPhone sales after granting Qualcomm an injunction against Apple - WTVA (0 visits)
Original#356: China bans most iPhone sales after granting Qualcomm an injunction against Apple
Number of duplicates (SimHash): 2

Dup#308: Foxconn Technology : China's Henan pledges to further open auto, aircraft sectors to foreign investors | MarketScreener
Original#368: Foxconn Technology : China's Henan pledges to further open auto, aircraft sectors to foreign investors
Number of duplicates (SimHash): 1

Dup#395: Apple Hit With iPhone Sales Ban in China, Qualcomm Says -- 4th Update
Original#370: Apple : Hit With iPhone Sales Ban in China, Qualcomm Says -- 4th Update
Number of duplicates (SimHash): 1

Dup#565: Top Apple Suppliers Post Solid Sales as iPhone Concerns Persist -
Original#371: Top Apple Suppliers Post Solid Sales as iPhone Concerns Persist
Number of duplicates (SimHash): 1

Dup#1372: The World’s Biggest Hedge Fund Is Getting Whacked, And Why “Moneyness” Matters | Economy
Dup#2304: The World's Biggest Hedge Fund Is Getting Whacked, And Why "Moneyness" Matters | Zero Hedge
Dup#1615: The World's Biggest Hedge Fund Is Getting Whacked, And Why "Moneyness" Matters
Dup#796: The World's Biggest Hedge Fund Is Getting Whacked, And Why 'Moneyness' Matters
Original#567: The World’s Biggest Hedge Fund Is Getting Whacked, And Why “Moneyness” Matters
Number of duplicates (SimHash): 4

Dup#561: Microsoft, Apple and others criticize Australian Government's new encryption law
Original#571: Microsoft, Apple and others criticize Australian Government’s new encryption law
Number of duplicates (SimHash): 1

Dup#437: China court bans iPhone sales
Original#592: Chinese court bans iPhone sales
Number of duplicates (SimHash): 1

Dup#607: Apple launches payments service in Germany following delay | News | WTVB
Original#601: Apple launches payments service in Germany following delay

Dup#2883: An Apple Watch told a 46-year-old man he had an irregular heartbeat. It was right
Original#865: An Apple Watch told a 46-year-old man he had an irregular heartbeat. It was right.
Number of duplicates (SimHash): 1

Dup#932: The Huawei arrest puts a ‘bullseye’ on Apple’s back at a potentially dangerous time for iPhone sales in China
Dup#1036: Tech: The Huawei arrest puts a 'bullseye' on Apple's back at a potentially dangerous time for iPhone sales in China
Original#870: The Huawei arrest puts a 'bullseye' on Apple's back at a potentially dangerous time for iPhone sales in China
Number of duplicates (SimHash): 2

Dup#5140: Apple iPhone 6, 16GB, space gray, unlocked (Merced) $155
Original#871: Apple iPhone 6, 16GB, space gray, unlocked (Merced) $160
Number of duplicates (SimHash): 1

Dup#527: China Bans The Sale Of Iphone 6S, 6S Plus, 7, 7 Plus, 8, 8 Plus And X - Phones
Original#877: China Bans The Sale Of Iphone 6S, 6S Plus, 7, 7 Plus, 8, 8 Plus And X
Number of duplicates (SimHa

Dup#857: The Galaxy S10+ Will Have a Headphone Jack, Turning It Into a Luxury Feature
Original#1075: The Galaxy S10 Will Have a Headphone Jack, Turning It Into a Luxury Feature
Number of duplicates (SimHash): 1

Dup#823: RELEASE BLITZ - Christmas Snacks Anthology
Dup#1098: Release Blitz: Christmas Snacks.
Dup#936: Release Blitz ***Christmas Snacks***
Dup#1212: Release Blitz ~ Christmas Snacks Anthology
Dup#905: ~*RELEASE BLITZ*~ Christmas Snacks Anthology
Dup#1299: Release Blitz: Christmas Snacks
Original#1092: *~Release Blitz: Christmas Snacks~*
Number of duplicates (SimHash): 6

Dup#823: RELEASE BLITZ - Christmas Snacks Anthology
Dup#1092: *~Release Blitz: Christmas Snacks~*
Dup#936: Release Blitz ***Christmas Snacks***
Dup#1212: Release Blitz ~ Christmas Snacks Anthology
Dup#905: ~*RELEASE BLITZ*~ Christmas Snacks Anthology
Dup#1299: Release Blitz: Christmas Snacks
Original#1098: Release Blitz: Christmas Snacks.
Number of duplicates (SimHash): 6

Dup#723: Apple accused by Egypt of v

...

Dup#4790: Apple Plans New $11bn Investment In U.S
Original#4637: Apple Plans New $11bn Investment In U.S.
Number of duplicates (SimHash): 1

Dup#4706: Apple To Build 1$bn Campus In Texas
Original#4652: Apple To Build $1bn Campuses In Texas
Number of duplicates (SimHash): 1

Dup#5439: You don’t have to love AirPods to love Apple — there are alternatives
Original#4656: You don't have to love AirPods to love Apple — there are alternatives
Number of duplicates (SimHash): 1

Dup#3670: U.S. tribunal to review ruling on Qualcomm request for iPhone ban | News | WIBQ
Dup#3521: U.S. tribunal to review ruling on Qualcomm request for iPhone ban
Dup#3542: US tribunal to review ruling on Qualcomm request for iPhone ban
Original#4657: U.S. ITC tribunal to review ruling on Qualcomm request for iPhone ban
Number of duplicates (SimHash): 3

Dup#4994: $1B Apple campus coming to Austin
Original#4659: $1B Apple campus coming to North Austin
Number of duplicates (SimHash): 1

Dup#4663: Free Limited Time App

Dup#4964: Apple to Inject $1 Billion Into New Campus in Austin
Original#4919: Apple to Inject $1 Billion Into New Campus in Austin, Texas
Number of duplicates (SimHash): 1

Dup#4205: Mirror for Roku TV 2.5 macOS
Original#4927: Mirror for Roku TV 2.5
Number of duplicates (SimHash): 1

Dup#4741: Apple Announces Plan to Build $1 Billion Austin Campus
Original#4934: Apple announces plan to build new $1 billion campus in Austin
Number of duplicates (SimHash): 1

Dup#4875: Tempow’s new Bluetooth profile lets you create AirPods clones more easily
Dup#5084: #USA Tempow’s new Bluetooth profile lets you create AirPods clones more easily
Original#4943: Tempow’s new Bluetooth profile lets you create AirPods clones more easily – TechCrunch
Number of duplicates (SimHash): 2

Dup#1581: Apple Watch series 3 (El reno) $280
Dup#4855: Apple Watch Series 3 $250
Original#4958: Apple Watch Series 3 $180
Number of duplicates (SimHash): 2

Dup#4919: Apple to Inject $1 Billion Into New Campus in Austin, Texas


Dup#5635: Apple deepens Austin ties, expands in other tech hotbeds - KVIA
Original#5190: Apple deepens Austin ties, expands in other tech hotbeds
Number of duplicates (SimHash): 1

Dup#5157: Qualcomm to file suits in Chinese courts to ban iPhone XS, XR sales - Nasdaq.com
Dup#5171: Apple : Qualcomm to file suits in Chinese courts to ban iPhone XS, XR sales
Original#5202: Apple : Qualcomm to file suits in Chinese courts to ban iPhone XS, XR sales | MarketScreener
Number of duplicates (SimHash): 2

Dup#4729: Apple deepens Austin ties, expands U.S. operations east and west
Original#5215: Apple deepens Austin ties, expands operations east and west - Thu, 13 Dec 2018 PST
Number of duplicates (SimHash): 1

Dup#5178: Apple quoted me $1,500 to repair a MacBook Pro, so I paid less than $500 at an 'unauthorised' Apple repair shop instead
Dup#5118: Apple quoted me $1,500 to repair a MacBook Pro, so I paid less than $500 at an 'unauthorized' Apple repair shop instead
Original#5216: Apple quoted me 

Dup#5073: iPhone XS specs vs. XS Max, XR, X: What's the same and different - CNET
Original#5393: iPhone XS specs vs. XS Max, XR, X: What's the same and different
Number of duplicates (SimHash): 1

Dup#5562: Apple To Open New Campus In Austin, Texas
Original#5402: Apple to open $1 billion campus in Austin, Texas
Number of duplicates (SimHash): 1

Dup#5711: Apple’s original TV shows and series: J.J. Abrams to produce ‘My Glory Was I Had Such Friends’ with Jennifer Garner
Original#5408: Apple’s original TV shows and series: J.J. Abrams to produce 'My Glory Was I Had Such Friends' with Jennifer Garner
Number of duplicates (SimHash): 1

Dup#5701: The AI Eye: NICE ( $NICE) Unveils New Version of #AI Automated Robotic Process Suite, Wipro Launches Automotive Innovation Center in Detroit and Apple ( $AAPL) to Invest $1 Billion on Austin Campus
Original#5412: Investorideas.com - The AI Eye: NICE ( $NICE) Unveils New Version of #AI Automated Robotic Process Suite, Wipro Launches Automotive Innov

Dup#4564: Work and Create Anywhere with UAG's Latest iPad Pro Cases
Original#5561: Work and Create Anywhere with UAG’s Latest iPad Pro Cases
Number of duplicates (SimHash): 1

Dup#5402: Apple to open $1 billion campus in Austin, Texas
Original#5562: Apple To Open New Campus In Austin, Texas
Number of duplicates (SimHash): 1

Dup#5498: Australia's anti-encryption law is so unpopular, there was only 1 comment in support and 342 against — and now the head of its spy agency is defending the law (AAPL, MSFT, FB)
Dup#5574: Australia's anti-encryption law is so unpopular, there was only 1 comment in support and 342 against - and now the head of its spy agency is defending the law (AAPL, MSFT, FB)
Dup#5558: Australia’s anti-encryption law is so unpopular, there was only 1 comment in support and 342 against — and now the head of its spy agency is defending the law (AAPL, MSFT, FB)
Original#5567: Australia’s anti-encryption law is so unpopular, there was only 1 comment in support and 342 against

In [26]:
# Clean up deduplicated dataset
feeds_unique_deduped = []
for i, feed in enumerate(feeds_unique):
    if str(feed['id']) not in dups:
        feeds_unique_deduped.append(feed)
len(feeds_unique_deduped)
len(dups)

787

In [27]:
def vec_similarity(input1, input2, vectors):
    term_vectors = [np.zeros(300), np.zeros(300)]
    terms = [input1, input2]
        
    for index, term in enumerate(terms):
        for i, t in enumerate(term.split(' ')):
            try:
                term_vectors[index] += vectors[t]
            except:
                term_vectors[index] += 0
        
    result = (1 - spatial.distance.cosine(term_vectors[0], term_vectors[1]))
    if result is np.nan:
        result = 0
        
    return result

In [31]:
count_dup = 0
for dupi in dup_indices:
    try:
        score = vec_similarity(feed_sel['title'], feeds_unique_deduped[int(dupi)]['title'], model_word2vec)
#         print(score)
    except:
        score = 0
    if score > 0.8:
        count_dup += 1
        print(feeds_unique_deduped[int(dupi)]['title'])
print("Number of Dupies (SimHash + WordVectors): " + str(count_dup))

Number of Dupies (SimHash + WordVectors): 0


In [32]:
# Save as txt file and print deduplicated subset of id, titles and published dates
# dup_indices_set = set(dup_indices)
with open('./deduped_apple.txt', 'w') as f:
    for feed in feeds:
        if str(feed['id']) not in dups:
#             print(' '.join((str(feed['id']), str(feed['title']),  feed['published'])))
            f.write(' '.join((str(feed['id']), str(feed['title']),  feed['published'], '\n')))

In [33]:
#3. Named Entity Recognition

In [34]:
def sendFile(input_data, headers, output_dir='./'):
#     with open(file_name, 'rb') as input_data:
    response = requests.post(calais_url, data=input_data, headers=headers, timeout=80)
    print ('status code: %s' % response.status_code)
    content = response.text
#     print ('Results received: %s' % content)
    if response.status_code == 200:
        saveFile('final_text_apple', output_dir, content)
        return response
            
def saveFile(file_name, output_dir, content):
    output_file_name = os.path.basename(file_name) + '.json'
    output_file = open(os.path.join(output_dir, output_file_name), 'wb')
    output_file.write(content.encode('utf-8'))
    output_file.close()

In [42]:
feeds_unique_deduped
RANDOM_INDEX = 7
print(feeds_unique_deduped[RANDOM_INDEX]['title'])
access_token = 'ZR2BvCL0pxKthAeKa0FKPbb9I9At9ows'
headers = {'X-AG-Access-Token' : access_token, 
           'Content-Type' : 'text/raw', 
           'outputformat' : 'application/json'
          }
input_data = feeds_unique_deduped[RANDOM_INDEX]['title'].encode('utf-8')
response = sendFile(input_data, headers)
tagging_result = json.loads(response.content)

Apple’s W1 chip: What is it, and why don’t Android users get a bite?
status code: 200


In [43]:
def extract_entity(tagging_result):
    for hash in tagging_result:        
        if '_typeGroup' in tagging_result[hash] and tagging_result[hash]['_typeGroup'] == 'entities':
#             print(tagging_result[hash])
            print(tagging_result[hash]['_type'], ':', 
                  tagging_result[hash]['name'], ', relevance: ', 
                  tagging_result[hash]['relevance'])
extract_entity(tagging_result)

Technology : W1 chip , relevance:  0.8
OperatingSystem : Android , relevance:  0.8
Technology : Android , relevance:  0.8


In [231]:
#4. Topic Clusters
# feeds_unique_deduped

In [37]:
def tokenize_titles(title):
    tokens = nltk.word_tokenize(title)
    lmtzr = WordNetLemmatizer()
    filtered_tokens = []
    
    for token in tokens:
        token = token.replace("'s", " ").replace("n’t", " not").replace("’ve", " have")
        token = re.sub(r'[^a-zA-Z0-9 ]', '', token)
        if token not in stopwords:
            filtered_tokens.append(token.lower())
    
    lemmas = [lmtzr.lemmatize(t,'v') for t in filtered_tokens]

    return lemmas

In [58]:
def clstr_lda(num_topics, titles):
    # top words to be identified
    n_top_words = 10

    tf_vectorizer = CountVectorizer(max_df=0.9, min_df=0.01, max_features=1000,
                                    tokenizer=tokenize_titles, ngram_range=(3,4))

    tf = tf_vectorizer.fit_transform(titles)

    lda = LatentDirichletAllocation(n_components=num_topics, max_iter=1000,
                                    learning_method='batch', learning_offset=10.,
                                    random_state = 1)
    lda.fit(tf)
    tf_feature_names = tf_vectorizer.get_feature_names()

    # print top topic words
    topics = dict()
    for topic_idx, topic in enumerate(lda.components_):
        topics[topic_idx] = [tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        print("Topic #%d:" % topic_idx)
        print(" | ".join([tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
    return topics

In [50]:
feed_titles = []
count = 0
for feed in feeds_unique_deduped:
#    if 'watch' not in feed['title'].lower():
        count += 1
#         print(feed['title'])
        feed_titles.append(str(feed['title']))
print(count)
print("Total number of titles: " + str(len(feed_titles)))

5022
Total number of titles: 5022


In [53]:
feed_titles[1000:1200]

['Apple Watch Series 4 Heart Rate app takes ‘faster reading with higher fidelity’ using Digital Crown',
 'Andrey Lezhnev is Prepared to Take M-1 Challenge FW Champ Nate Landwehr’s Title | Full Contact Fighter',
 'SAMSUNG SGH A886 DRIVER',
 'Tech News Flash',
 'Australia anti-encryption law rushed to passage',
 'Super Micro says external security audit found no evidence of backdoor chips',
 'National App Day: The App (on the Enterprise) You Can’t Live Without | MarketScreener',
 'Apple’s (AAPL) “Buy” Rating Reaffirmed at Tigress Financial',
 'Complaints about metal pieces in Jimmy Dean sausage prompt recall',
 'Big Tech Company Employees Give 90 Percent Of Political Donations To Democrats, According To New Poll',
 '2018 Chevrolet Silverado 3500HD LT (St. Cloud, MN) $38988',
 'China Has Not Banned iPhones, But The US Private Sector is Already in Panic Mode',
 'World of Warcraft 8.1 now defaults to DX12, expect a performance boost',
 'Qualcomm secures "ban" on iPhone sales in China',
 'Go

In [59]:
topics = clstr_lda(6, feed_titles)

Topic #0:
 apple  |    |  aapl  | campus austin  |  1 billion | apple watch series | 1 billion campus |  1 billion campus
Topic #1:
   |  apple  | apple watch series | campus austin  |  aapl  |  1 billion | 1 billion campus |  1 billion campus
Topic #2:
 1 billion | campus austin  |  aapl  |  apple  |    | apple watch series | 1 billion campus |  1 billion campus
Topic #3:
 aapl  | campus austin  |  1 billion |  apple  |    | apple watch series | 1 billion campus |  1 billion campus
Topic #4:
apple watch series |    | campus austin  |  aapl  |  apple  |  1 billion | 1 billion campus |  1 billion campus
Topic #5:
1 billion campus |  1 billion campus |  1 billion | campus austin  |  aapl  |  apple  |    | apple watch series


In [ ]:
#5. Topic Taxonomy

In [252]:
nlp = en_core_web_sm.load()
SUBJECTS = ["nsubj", "nsubjpass", "csubj", "csubjpass", "agent", "expl"]
OBJECTS = ["dobj", "dative", "attr", "oprd"] 

In [274]:
for feed in feeds_unique_deduped[:1]:
    selected_feed = feed['text']
#     print(selected_feed) 
    sentences = sent_tokenize(selected_feed)
    for sentence in sentences:
        print(sentence)
        print()

So is this it — the end of the decade-long bull market?

Or is it the sort of normal, healthy pullback that often follows a strong showing?

Last week, it felt like the bottom dropped out of the stock market.

The blue-chip Dow Jones industrial average fell 4.5 percent for the week, ending with a 559-point rout Friday that left the index in negative territory for the year.

The benchmark Standard & Poor's 500-stock index has fallen into a "correction," meaning it's off more than 10 percent from its peak reached Sept. 20.

And once-highflying tech stocks faltered with Apple, Facebook and Google parent Alphabet all ending last week showing stock-price losses for the year — driving the Nasdaq composite index into a correction too.

So is this it — the end of the decade-long bull market?

Or is it the sort of normal, healthy pullback that often follows a strong showing?

"Everyone is trying to figure that out," said Sam Stovall, chief investment strategist at CFRA Research, though he's amo

In [275]:
def getSubsFromConjunctions(subs):
    moreSubs = []
    for sub in subs:
        # rights is a generator
        rights = list(sub.rights)
        rightDeps = {tok.lower_ for tok in rights}
        if "and" in rightDeps:
            moreSubs.extend([tok for tok in rights if tok.dep_ in SUBJECTS or tok.pos_ == "NOUN"])
            if len(moreSubs) > 0:
                moreSubs.extend(getSubsFromConjunctions(moreSubs))
    return moreSubs

In [276]:
def getObjsFromConjunctions(objs):
    moreObjs = []
    for obj in objs:
        # rights is a generator
        rights = list(obj.rights)
        rightDeps = {tok.lower_ for tok in rights}
        if "and" in rightDeps:
            moreObjs.extend([tok for tok in rights if tok.dep_ in OBJECTS or tok.pos_ == "NOUN"])
            if len(moreObjs) > 0:
                moreObjs.extend(getObjsFromConjunctions(moreObjs))
    return moreObjs

In [277]:
def getVerbsFromConjunctions(verbs):
    moreVerbs = []
    for verb in verbs:
        rightDeps = {tok.lower_ for tok in verb.rights}
        if "and" in rightDeps:
            moreVerbs.extend([tok for tok in verb.rights if tok.pos_ == "VERB"])
            if len(moreVerbs) > 0:
                moreVerbs.extend(getVerbsFromConjunctions(moreVerbs))
    return moreVerbs

In [278]:
def findSubs(tok):
    head = tok.head
    while head.pos_ != "VERB" and head.pos_ != "NOUN" and head.head != head:
        head = head.head
    if head.pos_ == "VERB":
        subs = [tok for tok in head.lefts if tok.dep_ == "SUB"]
        if len(subs) > 0:
            verbNegated = isNegated(head)
            subs.extend(getSubsFromConjunctions(subs))
            return subs, verbNegated
        elif head.head != head:
            return findSubs(head)
    elif head.pos_ == "NOUN":
        return [head], isNegated(tok)
    return [], False

In [279]:
def isNegated(tok):
    negations = {"no", "not", "n't", "never", "none"}
    for dep in list(tok.lefts) + list(tok.rights):
        if dep.lower_ in negations:
            return True
    return False

In [280]:
def findSVs(tokens):
    svs = []
    verbs = [tok for tok in tokens if tok.pos_ == "VERB"]
    for v in verbs:
        subs, verbNegated = getAllSubs(v)
        if len(subs) > 0:
            for sub in subs:
                svs.append((sub.orth_, "!" + v.orth_ if verbNegated else v.orth_))
    return svs

In [281]:
def getObjsFromPrepositions(deps):
    objs = []
    for dep in deps:
        if dep.pos_ == "ADP" and dep.dep_ == "prep":
            objs.extend([tok for tok in dep.rights if tok.dep_  in OBJECTS or (tok.pos_ == "PRON" and tok.lower_ == "me")])
    return objs

In [282]:
def getObjsFromAttrs(deps):
    for dep in deps:
        if dep.pos_ == "NOUN" and dep.dep_ == "attr":
            verbs = [tok for tok in dep.rights if tok.pos_ == "VERB"]
            if len(verbs) > 0:
                for v in verbs:
                    rights = list(v.rights)
                    objs = [tok for tok in rights if tok.dep_ in OBJECTS]
                    objs.extend(getObjsFromPrepositions(rights))
                    if len(objs) > 0:
                        return v, objs
    return None, None

In [283]:
def getObjFromXComp(deps):
    for dep in deps:
        if dep.pos_ == "VERB" and dep.dep_ == "xcomp":
            v = dep
            rights = list(v.rights)
            objs = [tok for tok in rights if tok.dep_ in OBJECTS]
            objs.extend(getObjsFromPrepositions(rights))
            if len(objs) > 0:
                return v, objs
    return None, None

In [284]:
def getAllSubs(v):
    verbNegated = isNegated(v)
    subs = [tok for tok in v.lefts if tok.dep_ in SUBJECTS and tok.pos_ != "DET"]
    if len(subs) > 0:
        subs.extend(getSubsFromConjunctions(subs))
    else:
        foundSubs, verbNegated = findSubs(v)
        subs.extend(foundSubs)
    return subs, verbNegated

In [285]:
def getAllObjs(v):
    # rights is a generator
    rights = list(v.rights)
    objs = [tok for tok in rights if tok.dep_ in OBJECTS]
    objs.extend(getObjsFromPrepositions(rights))

    #potentialNewVerb, potentialNewObjs = getObjsFromAttrs(rights)
    #if potentialNewVerb is not None and potentialNewObjs is not None and len(potentialNewObjs) > 0:
    #    objs.extend(potentialNewObjs)
    #    v = potentialNewVerb

    potentialNewVerb, potentialNewObjs = getObjFromXComp(rights)
    if potentialNewVerb is not None and potentialNewObjs is not None and len(potentialNewObjs) > 0:
        objs.extend(potentialNewObjs)
        v = potentialNewVerb
    if len(objs) > 0:
        objs.extend(getObjsFromConjunctions(objs))
    return v, objs

In [286]:
def findSVOs(tokens):
    svos = []
    verbs = [tok for tok in tokens if tok.pos_ == "VERB" and tok.dep_ != "aux"]
    for v in verbs:
        subs, verbNegated = getAllSubs(v)
        # hopefully there are subs, if not, don't examine this verb any longer
        if len(subs) > 0:
            v, objs = getAllObjs(v)
            for sub in subs:
                for obj in objs:
                    objNegated = isNegated(obj)
                    svos.append((sub.lower_, "!" + v.lower_ if verbNegated or objNegated else v.lower_, obj.lower_))
    return svos

In [287]:
def getAbuserOntoVictimSVOs(tokens):
    maleAbuser = {'he', 'boyfriend', 'bf', 'father', 'dad', 'husband', 'brother', 'man'}
    femaleAbuser = {'she', 'girlfriend', 'gf', 'mother', 'mom', 'wife', 'sister', 'woman'}
    neutralAbuser = {'pastor', 'abuser', 'offender', 'ex', 'x', 'lover', 'church', 'they'}
    victim = {'me', 'sister', 'brother', 'child', 'kid', 'baby', 'friend', 'her', 'him', 'man', 'woman'}

    svos = findSVOs(tokens)
    wnl = WordNetLemmatizer()
    passed = []
    for s, v, o in svos:
        s = wnl.lemmatize(s)
        v = "!" + wnl.lemmatize(v[1:], 'v') if v[0] == "!" else wnl.lemmatize(v, 'v')
        o = "!" + wnl.lemmatize(o[1:]) if o[0] == "!" else wnl.lemmatize(o)
        if s in maleAbuser.union(femaleAbuser).union(neutralAbuser) and o in victim:
            passed.append((s, v, o))
    return passed

In [288]:
def printDeps(toks):
    for tok in toks:
        print(tok.orth_, tok.dep_, tok.pos_, tok.head.orth_, [t.orth_ for t in tok.lefts], [t.orth_ for t in tok.rights])

def testSVOs(sentences):
    #nlp = English()
    for sentence in sentences:
        tok = nlp(sentence)
        svos = findSVOs(tok)
    #     printDeps(tok)
    #    assert set(svos) == {('i', '!have', 'assistance'), ('he', '!provide', 'support')}
        if svos:
            for svo in svos:
                print('S:{0}, V:{1}, O:{2}'.format((svo[0]), svo[1], svo[2]))
        else:
            print('No SVO')

In [289]:
testSVOs()

No SVO
S:that, V:follows, O:showing
No SVO
S:that, V:left, O:index
S:it, V:'s, O:percent
S:stocks, V:faltered, O:all
No SVO
S:that, V:follows, O:showing
S:everyone, V:figure, O:that
S:obscuring, V:are, O:trends
S:that, V:keeping, O:pressure
S:trends, V:sparking, O:trading
S:they, V:include, O:tariffs
S:they, V:include, O:battle
S:they, V:include, O:prices
S:they, V:include, O:bursts
S:prices, V:raising, O:rates
S:pullback, V:is, O:—
No SVO
No SVO
No SVO
S:it, V:was, O:loss
S:market, V:reached, O:levels
S:s&p, V:gained, O:percent
No SVO
No SVO
S:we, V:had, O:amount
S:year, V:!was, O:anomaly
No SVO
No SVO
S:s&p, V:lost, O:percent
S:s&p, V:lost, O:decline
No SVO
S:yields, V:locking, O:money
No SVO
No SVO
S:investors, V:pay, O:attention
No SVO
S:it, V:be, O:buy
S:s&p, V:trading, O:earnings
No SVO
No SVO
S:street, V:expected, O:growth
S:we, V:see, O:amount
No SVO


In [9]:
# model_path = './'
# def load_wordvec_model(modelName, modelFile, flagBin):
#     print('Loading ' + modelName + ' model...')
#     model = KeyedVectors.load_word2vec_format(model_path + modelFile, binary=flagBin)
#     print('Finished loading ' + modelName + ' model...')
#     return model

# model_word2vec = load_wordvec_model('Word2Vec', 'GoogleNews-vectors-negative300.bin', True)
# model_fasttext = load_wordvec_model('FastText', 'fastText_wiki_en.vec', False)

Loading Word2Vec model...
Finished loading Word2Vec model...


In [ ]:
# Develop own taxonomy

In [ ]:
topic_taxonomy = {
    "Business":
    {
        "Trading and Investing": "stock trade sale dow market invest acquire merge purchase subsidiary",
        "Company Corporation and Competition": "compete partnership rivalry win lose deal giveup",
        "Policy Change": "operation change organizational policy support transfer"
    },
    "Technology":
    {
        "Laptop": "microsoft mac pro air surface lenovo dell alienware asus apple",
        "Smart Phone": "iphone android nexus huawei xiaomi",
        "Intelligent Home Appliance": "alexa google home mini ipod ipad amazon"
    }
} 

In [ ]:
def vec_similarity(input1, input2, vectors):
    term_vectors = [np.zeros(300), np.zeros(300)]
    terms = [input1, input2]
        
    for index, term in enumerate(terms):
        for i, t in enumerate(term.split(' ')):
            try:
                term_vectors[index] += vectors[t]
            except:
                term_vectors[index] += 0
        
    result = (1 - spatial.distance.cosine(term_vectors[0], term_vectors[1]))
    if result is 'nan':
        result = 0
        
    return result

In [2]:
# function checks whether the input words are present in the vocabulary for the model
def vocab_check(vectors, words):
    
    output = list()
    for word in words:
        if word in vectors.vocab:
            output.append(word.strip())
            
    return output

In [3]:
# function takes an input string, runs similarity for each item in topic_taxonomy, sorts and returns top 3 results
def classify_topics(input, vectors):
    feed_score = dict()
    for key, value in topic_taxonomy.items():
        max_value_score = dict()
        for label, keywords in value.items():
            max_value_score[label] = 0
            topic = (key + ' ' + keywords).strip()
            max_value_score[label] += float(calc_similarity(input, topic, vectors))
            
        sorted_max_score = sorted(max_value_score.items(), key=operator.itemgetter(1), reverse=True)[0]
        feed_score[sorted_max_score[0]] = sorted_max_score[1]
    return sorted(feed_score.items(), key=operator.itemgetter(1), reverse=True)[:3]

In [60]:
# example using Word2Vec
output1 = classify_topics('The Impact of Artificial Intelligence on Business', model_word2vec)
print(output1)
# # example output using FastText, note for FastText, string needs to be non-capitalized
# output2 = classify_topics('White House Has Plan To Force Tillerson Out Replace With CIA Chief'.lower(), model_fasttext)
# print(output2)
    

NameError: name 'classify_topics' is not defined